In [1]:
from trl import PPOTrainer
from tqdm.auto import tqdm



W0926 08:53:42.611000 18564 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/opt/miniconda3/envs/trl-env/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
# ==============================================================================
#  1. LOAD MODELS AND TOKENIZERS
# ==============================================================================
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from trl import AutoModelForCausalLMWithValueHead
from sklearn.model_selection import train_test_split

# --- Define Model Paths ---
# The policy model is the base LLM we want to fine-tune with PPO.
#POLICY_MODEL_ID = "dbmdz/german-gpt2" 
#TODO: baseline=non-fine tuned model meta-llama/Meta-Llama-3-8B-Instruct
POLICY_MODEL_ID = "frhew/sigdial_ft_a2" 


# This is the path to your custom-trained "rules_heavy" reward model.
RM_PATH = "rm_out_rules_heavy_final"

# --- Load Policy and Reference Models ---
# The policy model is loaded with a value head for PPO training.
# policy_model = AutoModelForCausalLMWithValueHead.from_pretrained(POLICY_MODEL_ID)
# the above is loaded later with lora

# The reference model is a frozen copy of the original policy.
# --- Load the standard reference model (without LoRA) ---
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(
                POLICY_MODEL_ID,
                torch_dtype=torch.bfloat16, # <-- Use lower precision
)

# the above line takes 30min tot downlaod 7 shareds each close to 5GB



Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:


# --- Load Tokenizers ---
policy_tokenizer = AutoTokenizer.from_pretrained(POLICY_MODEL_ID, padding_side='left') # Add padding_side
reward_tokenizer = AutoTokenizer.from_pretrained(RM_PATH, padding_side='left')     # Add padding_side

print("All models and tokenizers loaded successfully.")

# Set padding token for both tokenizers if it's not already set.
#TODO assess which padding is required 
if policy_tokenizer.pad_token is None:
    policy_tokenizer.pad_token = policy_tokenizer.eos_token
if reward_tokenizer.pad_token is None:
    reward_tokenizer.pad_token = reward_tokenizer.eos_token

"""
# --- Load Tokenizers ---
policy_tokenizer = AutoTokenizer.from_pretrained(POLICY_MODEL_ID)
reward_tokenizer = AutoTokenizer.from_pretrained(RM_PATH)

print("All models and tokenizers loaded successfully.")

# Set padding token for the policy tokenizer if it's not already set.
if policy_tokenizer.pad_token is None:
    policy_tokenizer.pad_token = policy_tokenizer.eos_token
"""

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


All models and tokenizers loaded successfully.


'\n# --- Load Tokenizers ---\npolicy_tokenizer = AutoTokenizer.from_pretrained(POLICY_MODEL_ID)\nreward_tokenizer = AutoTokenizer.from_pretrained(RM_PATH)\n\nprint("All models and tokenizers loaded successfully.")\n\n# Set padding token for the policy tokenizer if it\'s not already set.\nif policy_tokenizer.pad_token is None:\n    policy_tokenizer.pad_token = policy_tokenizer.eos_token\n'

In [4]:
# --- Load Custom Reward Model ---
reward_model = AutoModelForSequenceClassification.from_pretrained(RM_PATH)

In [5]:
# --- Configure Hardware Device (CPU vs. Apple Silicon GPU) ---
# Check for the availability of Apple's Metal Performance Shaders (MPS) for GPU
# acceleration and set the global device accordingly.
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: mps


In [6]:
# Move the reward model to the correct device and set it to evaluation mode.
reward_model.to(device) # 'device' was set to "mps" in a previous cell
reward_model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [7]:
# ==============================================================================
#  2. PREPARE DATASET FOR PPO
# ==============================================================================
import pandas as pd
from datasets import Dataset

# Load the dataset containing the prompts for the policy model.
df = pd.read_csv("data/ordered_simplifications_with_rules.csv", index_col=0)

# The 'query' is the prompt given to the PPO model. 
# We'll use the original complex sentence as the prompt.
df.rename(columns={"original_sentence": "query"}, inplace=True)
# df = df.sample(n=64, random_state=42) #Uncomment for Sampled Data with low number of datapoints

In [8]:
#Create a train/eval set
train_df, eval_df = train_test_split(df, test_size=0.15, random_state=42)
print(f"Training set size: {len(train_df)}, Evaluation set size: {len(eval_df)}")

# For this demonstration, we'll use a small sample. For a full run, use the entire df.
ppo_dataset = Dataset.from_pandas(train_df)

Training set size: 9440, Evaluation set size: 1667


In [9]:
def format_query(example):
    """
    Applies the instruction template "Vereinfache diesen Satz:" 
    to each query in the dataset.
    """
    # The 'query' in the example is the original, complex sentence.
    example['query'] = f"Vereinfache diesen Satz: {example['query']}"
    return example

# Apply the formatting function to every example in the dataset.
# The .map() function processes each row and updates the 'query' column.
###applying this on EVAL happens later on
ppo_dataset = ppo_dataset.map(format_query)

# You can now check an example to see the new format
print("Example of a formatted query:")
print(ppo_dataset[0]['query'])

Map:   0%|          | 0/9440 [00:00<?, ? examples/s]

Example of a formatted query:
Vereinfache diesen Satz: Auch die 17 Jahre alte Greta Thunberg aus Schweden hielt am Dienstag eine Rede.


In [10]:
# Create the Hugging Face Dataset from a smaller, random sample.
# Using 1024-2048 prompts is often sufficient for a PPO run.
# Using a smaller, diverse set of prompts is often sufficient to train a robust policy, 
# as it provides enough variety to generate the experiences the model needs to learn from, 
# without the massive memory cost.
#4,096, 6,144
ppo_dataset = Dataset.from_pandas(train_df.sample(n=2048, random_state=42))

def tokenize_query(examples):
    """Tokenizes the 'query' column for the PPO trainer."""
    # Remove `padding="max_length"`. The PPOTrainer's dataloader will handle padding dynamically.
    #return policy_tokenizer(examples["query"], truncation=True, max_length=60)
    # """Tokenizes the 'query' column for the PPO trainer."""
    return policy_tokenizer(examples["query"], 
                            truncation=True, 
                            padding="max_length", 
                            max_length=40)

# Tokenize the dataset and format it for PyTorch.
ppo_dataset = ppo_dataset.map(tokenize_query, batched=True,
                              remove_columns=['final_simplification', 'applied_rules', 'uid', 'query'])
ppo_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/2048 [00:00<?, ? examples/s]

In [11]:
#print(ppo_dataset)
#print(ppo_dataset[1])

In [12]:
#ppo_dataset ## TODO remove unnecessary cols

# example:
# tokenized_dataset = dataset.map(tokenize_function, 
#                                 batched=True,
#                                 remove_columns=['final_simplification', 'applied_rules', 'uid', 'query']
#                                 )

In [13]:
# ==============================================================================
#  3. CONFIGURE PPO (WITH LORA)
# ==============================================================================

"""
Current PPO script is not performing hyperparameter tuning. 
It is running a single training session with one fixed set of parameters defined in your PPOConfig.

Unlike the standard Trainer used for your reward model, 
the PPOTrainer does not have a built-in .hyperparameter_search() method. 
You have to implement the search logic yourself with a loop.

"""
from trl import PPOConfig, PPOTrainer
from peft import LoraConfig

# --- Define the LoRA configuration ---
# This object defines the configuration for LoRA (Low-Rank Adaptation),
# a technique for efficient model fine-tuning.
lora_config = LoraConfig(
    # r: The rank of the LoRA matrices. This is the most important parameter.
    # It controls the number of trainable parameters. A higher rank means more
    # expressive power but also more parameters to train. Common values are 8, 16, 32.
    r=16,
    
    # lora_alpha: A scaling factor for the LoRA updates. It's like a learning
    # rate for the adapter layers. A common practice is to set alpha to twice the rank (2 * r).
    lora_alpha=32,
    
    # lora_dropout: Applies dropout regularization to the LoRA layers. This helps
    # prevent overfitting by randomly setting a fraction of adapter activations to zero.
    lora_dropout=0.05,
    
    # bias: Determines which bias parameters in the model are trained. "none" is a
    # common setting that freezes all bias terms and only trains the new LoRA weights.
    bias="none",
    
    # task_type: Specifies the type of model you are adapting. This is crucial for
    # the PEFT library to correctly identify and modify the right layers.
    # "CAUSAL_LM" is correct for GPT-style models used for text generation.
    task_type="CAUSAL_LM",
)


In [14]:
# # --- Load the Policy Model with PEFT config ---
# # TRL's special model class can directly accept a peft_config.
# # This ensures the model is created in the exact format the PPOTrainer expects.
policy_model = AutoModelForCausalLMWithValueHead.from_pretrained(
    POLICY_MODEL_ID,
    torch_dtype=torch.bfloat16, # <-- Use lower precision
    peft_config=lora_config  # <-- Pass the LoRA config here during loading
)

# Enable gradient checkpointing to trade a little computation for a lot of memory.
#policy_model.gradient_checkpointing_enable()





Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [15]:
# --- Define the PPOConfig  ---
# A recommended, stable set of hyperparameters for a single run without a full search.
config = PPOConfig(
    model_name=POLICY_MODEL_ID,
    # A learning rate of 1.4e-5 is a common and effective starting point for fine-tuning.
    learning_rate=1.4e-5,
    # The number of times you iterate over the collected PPO experiences in each optimization step.
    ppo_epochs=5, #default is 4
    # The number of prompts to collect before performing an optimization.
    batch_size=16, #32,
    # The PPO batch is split into smaller mini-batches for the update.
    mini_batch_size=4, #8,

    # --- MEMORY FIX 2: Add Gradient Accumulation ---
    # This will process 4 mini-batches before performing a model update.
    # Effective Batch Size = 4 (mini_batch_size) * 4 (accumulation_steps) = 16
    gradient_accumulation_steps=4,

    # Disables external logging integrations like WandB.
    log_with=None,
    ### --- FIX
    # Use the full KL penalty to ensure stability with LoRA-adapted modles. This prevents the negative KL divergence by applying the KL calculation more robustly.
    # Use a KL penalty to stop the model from deviating too far from the original.
    kl_penalty="full", #changed frol kl
    # A slightly lower target KL can improve stability and prevent the model from changing too drastically.
    target_kl=0.05,
    # --- Recommended additions for stability ---
    # Normalizes the reward scores, which is a key practice for stable PPO training.
    use_score_scaling=True,
    # The coefficient for the value function loss in the PPO update.
    vf_coef=0.1,
)


In [16]:
# # ==============================================================================
# #  4. INITIALIZE THE PPO TRAINER (WITH DATA COLLATOR)
# # ==============================================================================

# # --- 1. Initialize the PPOTrainer with all our components ---

# # THE FIX: Create a data collator that will dynamically pad each batch.
# # It uses the policy_tokenizer to know what padding token to use.
# data_collator = DataCollatorWithPadding(tokenizer=policy_tokenizer)

# ppo_trainer = PPOTrainer(
#     config=config,
#     model=policy_model,
#     ref_model=ref_model,
#     tokenizer=policy_tokenizer,
#     dataset=ppo_dataset,
#     data_collator=data_collator  # <-- NEW: Add the data collator here
# )

# # ... (The rest of your code for generation_kwargs and the training loop remains the same) ...

In [17]:
# --- Initialize the PPOTrainer ---
# Initialize the PPOTrainer with all our components.
ppo_trainer = PPOTrainer(
    config=config,
    model=policy_model,
    ref_model=ref_model,
    tokenizer=policy_tokenizer,
    dataset=ppo_dataset,
)

In [18]:
# ==============================================================================
#  4. THE PPO TRAINING LOOP
# ==============================================================================
from trl import PPOTrainer
from tqdm.auto import tqdm

# Generation settings for creating the simplified responses.
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": policy_tokenizer.eos_token_id,
    "max_new_tokens": 40, # Control the length of the simplification
}

# The main training loop
for epoch in range(config.ppo_epochs):
    for batch in tqdm(ppo_trainer.dataloader, f"PPO Epoch {epoch+1}"):
        
        # A. Get prompts (queries) from the batch.
        query_tensors = batch['input_ids']

        # B. Generate responses from the policy model.
        # THE FIX: Convert the 2D batch tensor into a list of 1D tensors.
        queries_list = [q for q in query_tensors]
        response_tensors = ppo_trainer.generate(queries_list, **generation_kwargs)
        batch['response'] = policy_tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

        # C. Score the responses with your custom reward model.
        # This is where the "Feedback" from your diagram happens.
        texts_to_score = batch['response']
        rewards = []
        with torch.no_grad():
            # Tokenize for the reward model
            inputs = reward_tokenizer(texts_to_score, return_tensors="pt", padding=True, truncation=True).to(device)
            # Get the raw score (logits)
            reward_logits = reward_model(**inputs).logits.squeeze(-1)
            # Store rewards as a list of PyTorch tensors.
            rewards = [r for r in reward_logits]
            
        # D. Perform the PPO optimization step.
        # This updates the policy model's weights based on the rewards, while
        # also calculating the KL penalty against the reference model.
        #stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        # Use the 'queries_list' you already created for the generate step
        stats = ppo_trainer.step(queries_list, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

print("PPO Training Finished!")

# Save the final tuned model.
ppo_trainer.save_pretrained("my_ppo_tuned_model_V26.09")
print("Model saved to 'my_ppo_tuned_model'")

PPO Epoch 1:   0%|          | 0/128 [00:00<?, ?it/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


PPO Epoch 2:   0%|          | 0/128 [00:00<?, ?it/s]

PPO Epoch 3:   0%|          | 0/128 [00:00<?, ?it/s]

PPO Epoch 4:   0%|          | 0/128 [00:00<?, ?it/s]

PPO Epoch 5:   0%|          | 0/128 [00:00<?, ?it/s]

PPO Training Finished!


/opt/miniconda3/envs/trl-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Model saved to 'my_ppo_tuned_model'


/opt/miniconda3/envs/trl-env/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1407: UserWarning: Cannot retrieve user information assuming you are running in offline mode.
  warnings.warn("Cannot retrieve user information assuming you are running in offline mode.")


In [19]:
train_df.info()
train_df.to_csv('train_df_26Sep.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 9440 entries, 6467 to 10863
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   query                 9440 non-null   object
 1   final_simplification  9440 non-null   object
 2   applied_rules         9440 non-null   object
dtypes: object(3)
memory usage: 295.0+ KB


In [20]:
sampled_train_df = ppo_dataset.to_pandas()
sampled_train_df.info()
train_df.to_csv('sampled_train_df_26Sep.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2048 entries, 0 to 2047
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   input_ids       2048 non-null   object
 1   attention_mask  2048 non-null   object
dtypes: object(2)
memory usage: 32.1+ KB


In [21]:
eval_df.info()
eval_df.to_csv('eval_df_26Sep.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 1667 entries, 6973 to 11012
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   query                 1667 non-null   object
 1   final_simplification  1667 non-null   object
 2   applied_rules         1667 non-null   object
dtypes: object(3)
memory usage: 52.1+ KB


In [22]:
eval_df

,query,final_simplification,applied_rules
uid,,,
6973,Sie können den Tumor auch mit Medikamenten bek...,Sie können den Tumor auch mit Medikamenten bek...,['normalize_verb_tense']
9309,"Das war schon das 3. Mal in einer Woche, dass ...","Das ist schon das 3 Mal in einer Woche, dass i...","['convert_word_to_number', 'normalize_verb_ten..."
3745,Im Fernsehen wird über die Wahl berichtet.,Man hat berichtet.,['convert_passive_to_active']
6153,"In Hollywood "" wurde als "" Beste Komödie "" aus...",Man hat ausgezeichnet.,['convert_passive_to_active']
2759,Schwere Vorwürfe gegen Opern-Sänger Placido Do...,Schwere Vorwürfe gegen Opern·Sänger Placido Do...,['split_compound']
...,...,...,...
939,Bundes-Kanzler Sebastian Kurz von der ÖVP woll...,Bundes-Kanzler Sebastian Kurz hat von der ÖVP ...,['normalize_verb_tense']
8991,Ab 1. Mai werden aber viele Sport-Plätze wiede...,Ab 1 Mai werden aber viele Sport·Plätze wieder...,"['convert_word_to_number', 'split_compound', '..."
12404,Auch gesungen werden darf nur im Freien oder m...,Man hat gesungen.,['convert_passive_to_active']


In [23]:
# # Define the generation settings used during training.
# generation_kwargs = {
#     "min_length": -1,
#     "top_k": 0.0,
#     "top_p": 1.0,
#     "do_sample": True,
#     "pad_token_id": policy_tokenizer.eos_token_id,
#     "max_new_tokens": 40,
# }

# # Define the function to score responses with your reward model.
# def compute_rewards_from_rm(responses: list[str]) -> torch.Tensor:
#     """
#     Computes rewards for a list of strings using the trained regression reward model.
#     """
#     with torch.no_grad():
#         # Tokenize the responses for the reward model.
#         inputs = reward_tokenizer(
#             responses,
#             return_tensors="pt",
#             padding=True,
#             truncation=True,
#         ).to(device)
        
#         # Get the reward model's output logits (the scalar reward values).
#         rewards = reward_model(**inputs).logits.squeeze(-1)
        
#     return rewards

# print("\n--- Setup complete. You can now run your evaluation script. ---")

In [24]:
# print("\n--- Evaluating and Logging Final Model Performance ---")

# # Take a small, fresh sample from your original dataframe for evaluation
# # Using a different random_state ensures we get a different set of prompts

# eval_df_small = eval_df.sample(n=50, random_state=49)
# eval_dataset = Dataset.from_pandas(eval_df_small)

# # Tokenize the evaluation prompts
# # Note: We are not using the dataloader here, just tokenizing a small batch
# eval_prompts = policy_tokenizer(
#     eval_df["query"].tolist(), # Use the pandas DataFrame here
#     return_tensors="pt",
#     padding=True,
#     truncation=True,
#     max_length = 60
# )
# eval_queries_list = [q.to(ppo_trainer.accelerator.device) for q in eval_prompts['input_ids']] # Ensure tensors are on the correct device

# # Generate responses with the FINAL trained model
# # We use torch.no_grad() for efficiency as we are not training
# with torch.no_grad():
#     eval_response_tensors = ppo_trainer.generate(eval_queries_list, **generation_kwargs)

# # THE FIX: Isolate the generated tokens before decoding
# # ==============================================================================
# generated_tokens = []
# # Get the original prompts' attention mask to find their true, unpadded lengths
# prompt_attention_mask = eval_prompts['attention_mask']

# for i, response_tensor in enumerate(eval_response_tensors):
#     # Find the actual length of the prompt by summing its attention mask
#     prompt_len = torch.sum(prompt_attention_mask[i])
    
#     # Slice the individual response tensor to get only the newly generated part
#     generated_part = response_tensor[prompt_len:]
#     generated_tokens.append(generated_part)

# # Decode only the generated part
# eval_responses = policy_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# # ==============================================================================

# # Score the final responses with the reward model
# eval_rewards = compute_rewards_from_rm(eval_responses)

# # Print the results in a clean format
# print("\n--- Final Model Generations ---")
# for i in range(len(eval_dataset["query"])):
#     print(f"Query:    {eval_dataset['query'][i]}")
#     print(f"Response: {eval_responses[i]}")
#     print(f"Reward:   {eval_rewards[i].item():.4f}")
#     print("-" * 50)